In [1]:
#pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [118]:
from selenium import webdriver
import pandas as pd

driver_path = r'C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe'

def run(driver_path):
    
    # The result will be stored in a list of lists (later converted to pandas dataframe). 
    # This is a very generic approach because no one told us what the result will
    # be used for. Otherwise different data sctucture could be used.
    result = []
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})    
    
    # Wrapping everything in "try ... finally" just to close and quit driver
    # every time something goes wrong. No need to catch exceptions here.
    try:
        driver = webdriver.Chrome(executable_path=driver_path, chrome_options=chrome_options)
        driver.maximize_window()
        driver.get('https://tproger.ru/quiz/real-programmer/')

        button = driver.find_element_by_class_name('quiz_button')
        button.click()
    
        q_num = int(driver.find_element_by_class_name('tquiz_progress')\
            .text.split(' ')[2])
        
        for q in range(0, q_num):
            
            quiz_item = driver.find_element_by_xpath(f'//div[@class="quiz_item" and @data-num="{q}"]')
            question = quiz_item.find_element_by_class_name('question').text

            # We don't care which answer to click - let's just find the first one
            answer = quiz_item.find_element_by_class_name('answer')
            answer.click()

            answers = quiz_item.find_elements_by_class_name('answer')   

            # leaving only answer text (without the comments)
            right_answer = quiz_item.find_element_by_class_name('correct_answer')\
                .text.split('\n')[0]

            for a in answers:
                try:
                    # leaving only answer text (without the comments)
                    text = a.find_element_by_class_name('answer_text')\
                        .text.split('\n')[0]

                    if text == right_answer:
                        is_correct = True
                    else:
                        is_correct = False

                    # percentage of people that gave this answer (cast to integer)
                    percent = int(a.find_element_by_class_name('percent')\
                        .text.replace('%',''))

                except ValueError:
                    # There are other elements on the page with class 'answer'.
                    # They don't contain any valuable information. Whenever
                    # one of this element is reached ValueError will be thrown 
                    # beacuse int('') is an invalid operation. Let's just
                    # break out of the inner loop (answers) when that happens 
                    # and go on with the outer loop (questions)
                    break

                #print(f"{q} ; {question} ; {text} ; {is_correct} ; {percent}")     
                result.append([q, question, text, is_correct, percent])

            button = driver.find_element_by_xpath(f'//button[@data-current="{q}"]')
            button.click()                         
        
    finally:
        driver.close()
        driver.quit()
        return pd.DataFrame(result, columns=['question_num', 'question', 'answer', 'is_correct', 'percent'])
    
result = run(driver_path)

<ipython-input-118-de617410a65b>:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_path, chrome_options=chrome_options)


In [119]:
result

,question_num,question,answer,is_correct,percent
0,0,Начнём с простого. Какую первую программу обыч...,Hello world,True,90
1,0,Начнём с простого. Какую первую программу обыч...,Сортировку «пузырьком»,False,3
2,0,Начнём с простого. Какую первую программу обыч...,Для взлома аккаунта «ВКонтакте». Такая програм...,False,2
3,0,Начнём с простого. Какую первую программу обыч...,Это вопрос с подвохом. Начинают они с починки ...,False,5
4,1,"Бывает ли так, что программа скомпилировалась ...","Да, конечно",False,25
5,1,"Бывает ли так, что программа скомпилировалась ...","Нет, это фантастика",True,75
6,2,"Представим гипотетическую ситуацию, в которой ...",Выключу комп и спокойно пойду спать — дело сде...,False,5
7,2,"Представим гипотетическую ситуацию, в которой ...",Порадуюсь за себя и продолжу писать код,False,18
8,2,"Представим гипотетическую ситуацию, в которой ...","Буду искать ошибку в компиляторе, где-то она д...",True,76
9,3,Движемся дальше. Вопрос на проверку логическог...,"Изменятся, конечно",True,89
